# SQL intermediate concepts
## Modifying Data
- INSERT
- UPDATE
- DELETE
- ALTER TABLE
- pragma table_info(tablename) : describes data type of columns

In [1]:
import sqlite3
conn=sqlite3.connect('factbook.db')

### Selecting by date

In [27]:
date = conn.execute('select * from facts where updated_at > "2015-10-30 16:00" and updated_at < "2015-11-2 15:00";').fetchall()
print date[:2]

[(1, u'af', u'Afghanistan', 652230, 652230, 0, 32564342, 2.32, 38.57, 13.89, 1.51, u'2015-11-01 13:19:49.461734', u'2015-11-01 13:19:49.461734', None), (2, u'al', u'Albania', 28748, 27398, 1350, 3029278, 0.3, 12.92, 6.58, 3.3, u'2015-11-01 13:19:54.431082', u'2015-11-01 13:19:54.431082', None)]


In [31]:
pragma table_info(facts)

SyntaxError: invalid syntax (<ipython-input-31-ff27364874b1>, line 1)

### Insert
Can also insert NULL values

In [32]:
conn.execute(""" insert into facts
values (262, "dq", "DataquestLand", 60000, 40000, 20000, 500000, 100, 50, 10, 20, 
"2016-02-25 12:00:00", "2016-02-25 12:00:00") """);

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 46))



OperationalError: table facts has 14 columns but 13 values were supplied

### Update
update 'table_name' set var = 'to rename' where var = 'want to rename'

In [ ]:
conn.execute('update facts set name="DataquestLand" where name="United States";')

### Delete
delete from 'table_name' where var = 'to delete'

In [ ]:
conn.execute('delete from facts where name = "Canada";')

### Alter Table
alter table 'table_name' add 'column to add' 'column data type'

In [ ]:
conn.execute('alter table facts add leader text;')

## Create Table

![table](createtable.png)

## Creating Relations Table
![table](relationstable.png)

- foreign key('col name') : column we want to create that will have relations with another table
- references 'table_name'('col name') : reference of relation

**references are one-sided**

In [2]:
conn.execute("""create table factbook.states(
    id integer primary key,
    name text,
    area integer,
    country integer,
    foreign key(country) references facts(id))""")

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 46))



OperationalError: unknown database factbook

### Querying across Foreign Keys
SELECT * from states INNER JOIN facts ON states.country == facts.id;

- JOIN : table to join with
    + INNER JOIN : only displays rows where there is a match for the ON condition in both tables. Any rows that aren't matched are excluded.
    + LEFT OUTER JOIN : if there is no match for a row from the table on the left side of the ON condition, it is shown with NULL values for all the right side values.
    + RIGHT OUTER JOIN : if there is no match for a row from the table on the right side of the ON condition, it is shown with NULL values for all the left side values
- ON : specifying which index is used to join merge

# Database Normalization

Most common type of data relations:
- one-to-many
- many-to-many

### **One-to-many example:**  
![norm](normalization.png)

In [ ]:
portman_movies=conn.execute("""SELECT ceremonies.year, nominations.movie FROM ceremonies 
INNER JOIN nominations
ON ceremonies.id==nominations.ceremony_id
WHERE nominations.nominee == 'Natalie Portman';""").fetchall()

print(portman_movies) #returns [(2010, 'Black Swan'), (2004, 'Closer')]

### **many-to-many relations need a join table**

![mm](manytomany.png)

example:

CREATE table movies_actors (  
id INTEGER PRIMARY KEY,  
movie_id INTEGER REFERENCES movies(id),  
actor_id INTEGER REFERENCES actors(id)   
);

In [ ]:
query = '''select actors.actor, movies.movie from movies
inner join movies_actors on movies.id == movies_actors.movie_id
inner join actors on movies_actors.actor_id == actors.id
where movies.movie == "The King's Speech"; '''

kings_actors = conn.execute(query).fetchall() 
# returns [('Colin Firth', "The King's Speech"), 
# ('Geoffrey Rush', "The King's Speech"), ('Helena Bonham Carter', "The King's Speech")]